In [299]:
import pandas as pd
import datetime
from ipywidgets import *
pd.set_option('display.max_colwidth', 128)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 60)

In [300]:
"""
-- Modified original query from https://dune.com/queries/92408/184718

SELECT 
  tx.hash,
  tx.success,
  --pid."name", 
  mints."_projectId" AS ProjectID,
  tx.value/1e18 AS price_eth,
  date_trunc('second', mints."evt_block_time") AS time,   
  mints."_to" AS buyer, 
  (tx."gas_used" * tx."gas_price"/1e18) AS gas_eth
FROM artblocks."GenArt721_evt_Mint" mints -- old contrct
LEFT JOIN ethereum.transactions tx
  ON mints."evt_tx_hash" = tx."hash"
--LEFT JOIN dune_user_generated.ArtBlocksProjectIDs pid 
--  ON pid.id = mints."_projectId"

UNION ALL 
    
SELECT 
  tx.hash,
  tx.success,
  --pid."name", 
  mints."_projectId" AS ProjectID, 
  tx.value/1e18 AS price, 
  date_trunc('second', mints."call_block_time") AS time, 
  mints."_by" AS buyer, 
  (tx."gas_used" * tx."gas_price"/1e18) AS gas_eth
FROM artblocks."GenArt721Core_call_mint" mints -- new contract
LEFT JOIN ethereum.transactions tx
  ON mints."call_tx_hash" = tx."hash"
--LEFT JOIN dune_user_generated.ArtBlocksProjectIDs pid 
--  ON pid.id = mints."_projectId"
WHERE "output__tokenId" is not null
ORDER BY time DESC
"""

d = pd.read_csv('../mint.csv')
d["time"] = pd.to_datetime(d["time"])
display(d.dtypes)

# sort by time and descending gas for most probable execution order without looking at transaction order numbers.
d.sort_values(by=["time", "gas_eth"], ascending=[True, False], inplace=True)

d.head()

hash                      object
success                     bool
projectid                  int64
price_eth                float64
time         datetime64[ns, UTC]
buyer                     object
gas_eth                  float64
dtype: object

,hash,success,projectid,price_eth,time,buyer,gas_eth
204059,\xc86f9caf0307f66d63c03aa1952f47e1a7f09243a8e7e3b26faa374a60c253dc,True,2,0.10,2020-11-27 15:58:01+00:00,\x7d42611012fdbe366bf4a0481fc0e1abf15e245a,0.015308
204058,\x2b4c7709bcb24f5f0337fcf5c045a5cbc911ab118f8bd0439db52615aa12d2ad,True,2,0.10,2020-11-27 16:00:31+00:00,\x7d42611012fdbe366bf4a0481fc0e1abf15e245a,0.013010
204057,\x99666b7a136f58b78abbb8226bae746e51293c927d2efc8d3d2e4bb4f3c1f500,True,2,0.10,2020-11-27 16:08:37+00:00,\x7d42611012fdbe366bf4a0481fc0e1abf15e245a,0.015034
204056,\x148da1d93e382e3220df987557f0240f14e24772feb4435a7a0337d607da6cdf,True,1,0.05,2020-11-27 16:10:41+00:00,\xc7391970d642faf65fabac8f63b0d41c4481d787,0.017108
204055,\x9e812dec2467b3f9c84fc3e589cc608360284a51def0bcae675fd43bae4da26f,True,2,0.10,2020-11-27 16:11:28+00:00,\xc7391970d642faf65fabac8f63b0d41c4481d787,0.014986


In [301]:
display(d.info())
display("Number of successful mints: ", d.success.sum())
d.describe(include=['bool','float', 'int', 'datetime'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204060 entries, 204059 to 0
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype              
---  ------     --------------   -----              
 0   hash       204060 non-null  object             
 1   success    204060 non-null  bool               
 2   projectid  204060 non-null  int64              
 3   price_eth  204060 non-null  float64            
 4   time       204060 non-null  datetime64[ns, UTC]
 5   buyer      204060 non-null  object             
 6   gas_eth    204060 non-null  float64            
dtypes: bool(1), datetime64[ns, UTC](1), float64(2), int64(1), object(2)
memory usage: 11.1+ MB


None

'Number of successful mints: '

198604

,success,projectid,price_eth,gas_eth
count,204060,204060.000000,204060.000000,204060.000000
unique,2,NaN,NaN,NaN
top,True,NaN,NaN,NaN
freq,198604,NaN,NaN,NaN
mean,NaN,152.841855,0.460313,0.074256
std,NaN,98.342955,0.942731,0.142770
min,NaN,0.000000,0.000000,0.000000
25%,NaN,74.000000,0.100000,0.016525
50%,NaN,143.000000,0.120000,0.038102
75%,NaN,227.000000,0.287963,0.074728


In [302]:
types = pd.read_csv('../typebyname.csv')
display(types.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   label         146 non-null    object 
 1   eth_total     146 non-null    float64
 2   usd_total     145 non-null    float64
 3   eth_original  146 non-null    float64
 4   refund_eth    146 non-null    float64
 5   usd_original  146 non-null    float64
 6   refund_usd    146 non-null    float64
 7   mint_count    146 non-null    int64  
 8   projectid     146 non-null    int64  
 9   project_type  146 non-null    object 
dtypes: float64(6), int64(2), object(2)
memory usage: 11.5+ KB


None

In [303]:
mints = d[d["success"]]

import statistics

def getMiddleValue(pdSeries):
    mid = (pdSeries.count() / 2).astype(int)
    return pdSeries.iloc[mid]

types = types.set_index("projectid")
types = types["project_type"]
types = types.reset_index()
# adding project type to mint dataset
mints = pd.merge(mints, types, on="projectid", how="outer")
print(mints[mints["projectid"] == 283])
mintsByProjectId = mints.groupby("projectid")
mintsByProjectId = pd.DataFrame({
    "count": mintsByProjectId["projectid"].count(),
    "firstMintTime": mintsByProjectId["time"].first(),
    "lastMintTime": mintsByProjectId["time"].last(),
    "lastMintPriceTotal": mintsByProjectId["price_eth"].last() + mintsByProjectId["gas_eth"].last(),
    "minMintPrice": mintsByProjectId["price_eth"].min(),
	"medianMintPrice": mintsByProjectId["price_eth"].median(),
	"meanMintPrice": mintsByProjectId["price_eth"].mean(),
    "medianMintTime":  mintsByProjectId["time"].apply(lambda x: getMiddleValue(x)),
    "projectType": mintsByProjectId["project_type"].first()
    })
mintsByProjectId["latterMintWindowInMins"] = round((mintsByProjectId["lastMintTime"] - mintsByProjectId["medianMintTime"]).dt.total_seconds() / 60, 2)
mintsByProjectId["totalMintWindowInMins"] = ((mintsByProjectId["lastMintTime"] - mintsByProjectId["firstMintTime"]).dt.total_seconds() / 60)
mintsByProjectId["latterMintWindowUnder4Hours"] = (mintsByProjectId["latterMintWindowInMins"] <= 240)

                                                                      hash success  projectid  price_eth                      time                                       buyer   gas_eth project_type
179726  \x2b1f18ffdb7e6cf77136838da35599042e4512e9a3cfafa7b9bfaf5e36c6f1b2    True        283      0.000 2022-04-02 19:04:42+00:00  \xff265b0151dc73fbeb172d9ff540eeeee756de87  0.016814      Curated
179727  \x2ca8330ea1e35cac91057020f5c7c2893496a65ebb3f011de246fed16ce3e1a3    True        283      1.500 2022-04-14 20:30:44+00:00  \xf25d986bd6514bcb0105ab6cf039c9b819c7fc67  0.020369      Curated
179728  \x5fe42748394554eecd5a295f193e7cdb0f5006db32da0d9424293d7dd62fb7e0    True        283      1.500 2022-04-14 20:31:10+00:00  \xf50123573c02abb33aa100d0ac1a49481f9e0edd  0.019855      Curated
179729  \xdfdcc78d612b76307f8f03d98af6d9a3bcc5a72b6b81b39c0bea424912037205    True        283      1.500 2022-04-14 20:31:10+00:00  \x8036bb84031e60a1bac93078c0e59bbd5e485db9  0.019855      Curated
179730  \x

In [304]:
mintsByProjectIdNoIndex = mintsByProjectId.reset_index()
mintsByProjectIdNoIndex = mintsByProjectIdNoIndex[mintsByProjectIdNoIndex["projectType"] == "Curated"]
settings = ["medianMintPrice", "meanMintPrice", "minMintPrice", "totalMintWindowInMins"]
def updateStats(i = 1):
	mintsByProjectIdNoIndex.plot.scatter(x="projectid", y=settings[i], figsize=(20,8), title="curated collections")
	mintsByProjectIdNoIndex[mintsByProjectIdNoIndex["latterMintWindowUnder4Hours"] == True].plot.scatter(x="projectid", y=settings[i], figsize=(20,8), title="latter mint window under 4 hours")

interact(updateStats)

interactive(children=(IntSlider(value=1, description='i', max=3, min=-1), Output()), _dom_classes=('widget-int…

<function __main__.updateStats(i=1)>

In [305]:
"""
-- Modified original query from https://dune.com/queries/160701/314169

select distinct block_time, 
  ROUND("nft_token_id"::numeric / 1000000) as projectid,
  round(eth_amount, 2) as eth_price, 
  usd_price, 
  link, 
  platform, 
  left(seller::text, 7) as seller, 
  left(buyer::text, 7) as buyer 
from 
(
select 
  block_time, 
  platform, 
  usd_amount, 
     
  case 
     when ("original_currency" = 'ETH' OR "original_currency" = 'WETH')
             THEN  ("original_amount")
    else 0  
  END as eth_amount, 
  "usd_amount" as usd_price,

   
 CONCAT('<a href="https://opensea.io/assets/', CONCAT('0x', substring(a."nft_contract_address"::text from 3)), '/', a.nft_token_id,  '/?ref=0x8F903cFC0Af3C2EC0d872c57538AF5e071544a57','" target="_blank" >', 'View on OS', '</a>') as  link,
   
 seller, 
 buyer, 
 tx_hash,
 nft_token_id

from nft."trades" a
WHERE 
     "trade_type" = 'Single Item Trade'
     AND (a.nft_contract_address = '\xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270'
    OR  a.nft_contract_address = '\x059edd72cd353df5106d2b9cc5ab83a52287ac3a')
ORDER BY block_time DESC 
) gg
-- WHERE block_time > '{{Date}}'
order by block_time DESC
"""

p = pd.read_csv('../sales.csv')

display("Before filtering:", len(d))

# cleaning up weird project ids
#p_removed = p[p["projectid"].str.len() >= 8]
#p = p[p["projectid"].str.len() < 8]

# casting
p["time"] = pd.to_datetime(p["time"])
p["projectid"] = p["projectid"].astype(int)

p.sort_values(by=["time"], ascending=[True], inplace=True)

print(p)
# adding derived data
p["normalized_price"] = p["eth_total"] / p["projectid"].map(mintsByProjectId.lastMintPriceTotal)
p["lastMintTime"] = p["projectid"].map(mintsByProjectId.lastMintTime)
p["isWithin2hFromLastMintTime"] = ((p["time"] - p["lastMintTime"]).dt.total_seconds() / 60 < 120) & ((p["time"] - p["lastMintTime"]).dt.total_seconds() > 0)

display(p.dtypes)
display(p.describe(include=['bool','float', 'int', 'datetime']))
p

'Before filtering:'

204060

                           time                  name  eth_total   usd_total  buyers    tokenid project_type  projectid platform
50313 2022-01-01 00:00:00+00:00  Organized Disruption      0.010    36.82120       1  133000016      Factory        133  OpenSea
50312 2022-01-01 00:00:00+00:00       Flowers by RVig      0.075   276.15900       1  116000553      Factory        116  OpenSea
50311 2022-01-01 00:02:00+00:00  Algobots by Stina Jo      1.750  6450.37750       1   40000287      Curated         40  OpenSea
50310 2022-01-01 00:04:00+00:00   Andradite by Eltono      0.001     3.68593       1   71000152      Factory         71  OpenSea
50309 2022-01-01 00:04:00+00:00  Skulptuur by Piter P      1.300  4791.70900       1  173000810      Curated        173  OpenSea
...                         ...                   ...        ...         ...     ...        ...          ...        ...      ...
4     2022-08-15 22:48:00+00:00  Autology by steganon        NaN         NaN       1  209000839  

time                          datetime64[ns, UTC]
name                                       object
eth_total                                 float64
usd_total                                 float64
buyers                                      int64
tokenid                                     int64
project_type                               object
projectid                                   int64
platform                                   object
normalized_price                          float64
lastMintTime                  datetime64[ns, UTC]
isWithin2hFromLastMintTime                   bool
dtype: object

,eth_total,usd_total,buyers,tokenid,projectid,normalized_price,isWithin2hFromLastMintTime
count,47742.000000,47743.000000,50314.000000,5.031400e+04,50314.000000,47742.000000,50314
unique,NaN,NaN,NaN,NaN,NaN,NaN,2
top,NaN,NaN,NaN,NaN,NaN,NaN,False
freq,NaN,NaN,NaN,NaN,NaN,NaN,42449
mean,1.524039,3323.371389,1.000795,2.248195e+08,224.818897,5.699134,NaN
std,6.073997,13864.160864,0.028185,9.017870e+07,90.179107,44.727424,NaN
min,0.000000,0.000000,1.000000,4.000000e+00,0.000000,0.000000,NaN
25%,0.140000,269.196853,1.000000,1.660004e+08,166.000000,0.631977,NaN
50%,0.330000,684.881600,1.000000,2.530010e+08,253.000000,1.205468,NaN
75%,1.100000,2407.660750,1.000000,2.870001e+08,287.000000,2.169821,NaN


,time,name,eth_total,usd_total,buyers,tokenid,project_type,projectid,platform,normalized_price,lastMintTime,isWithin2hFromLastMintTime
50313,2022-01-01 00:00:00+00:00,Organized Disruption,0.010,36.82120,1,133000016,Factory,133,OpenSea,0.035525,2021-08-11 20:34:28+00:00,False
50312,2022-01-01 00:00:00+00:00,Flowers by RVig,0.075,276.15900,1,116000553,Factory,116,OpenSea,0.444768,2021-08-02 17:06:52+00:00,False
50311,2022-01-01 00:02:00+00:00,Algobots by Stina Jo,1.750,6450.37750,1,40000287,Curated,40,OpenSea,6.797341,2021-04-10 17:05:09+00:00,False
50310,2022-01-01 00:04:00+00:00,Andradite by Eltono,0.001,3.68593,1,71000152,Factory,71,OpenSea,0.009643,2021-06-26 20:05:29+00:00,False
50309,2022-01-01 00:04:00+00:00,Skulptuur by Piter P,1.300,4791.70900,1,173000810,Curated,173,OpenSea,0.224433,2021-09-27 16:29:09+00:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...
4,2022-08-15 22:48:00+00:00,Autology by steganon,NaN,NaN,1,209000839,Curated,209,OpenSea,NaN,2021-11-22 19:02:08+00:00,False
3,2022-08-15 22:50:00+00:00,Talking Blocks by RE,NaN,NaN,1,55000438,Factory,55,OpenSea,NaN,2021-07-27 16:37:39+00:00,False
2,2022-08-15 22:58:00+00:00,Lava Glow by JEANVAS,NaN,NaN,1,110000430,Factory,110,OpenSea,NaN,2021-08-11 17:06:16+00:00,False
1,2022-08-15 23:39:00+00:00,Aithérios by Jorge L,NaN,NaN,1,196000074,Factory,196,OpenSea,NaN,2021-11-15 22:49:46+00:00,False


In [306]:
tradesByProjectId = p.groupby("projectid")
tradesByProjectId = pd.DataFrame({
    "tradeCount": tradesByProjectId["projectid"].count(),
    "tradeCount2hr": tradesByProjectId["isWithin2hFromLastMintTime"].sum(),
    "medianNormPrice2h": tradesByProjectId.apply(lambda df: df[df["isWithin2hFromLastMintTime"]].normalized_price.median()),
    "projectType": tradesByProjectId["project_type"].first(),
})

In [307]:
def update(projectid = 331):
    pId = p[p["projectid"] == projectid]
    display("project type: ", pId["project_type"].iloc[0])
    pId[pId["isWithin2hFromLastMintTime"]].plot(x="time", y=["normalized_price", "eth_total"], figsize=(20,8))

interact(update)

interactive(children=(IntSlider(value=331, description='projectid', max=993, min=-331), Output()), _dom_classe…

<function __main__.update(projectid=331)>

In [308]:
print(tradesByProjectId)
# removing 0 sales projects and weird ones
tradesByProjectId = tradesByProjectId[tradesByProjectId["tradeCount2hr"] > 0]
tradesByProjectId = tradesByProjectId.groupby("projectType")
averageTrades = pd.DataFrame({
    "averageTradeWithin2hr": tradesByProjectId["tradeCount2hr"].mean(),
    "minTradeWithin2hr": tradesByProjectId["tradeCount2hr"].min(),
    "maxTradeWithin2hr": tradesByProjectId["tradeCount2hr"].max(),
    "medianTradeWithin2hr": tradesByProjectId["tradeCount2hr"].median(),
})
averageTrades

           tradeCount  tradeCount2hr  medianNormPrice2h projectType
projectid                                                          
0                1534              4          38.113736     Curated
1                  51              0                NaN     Curated
2                  71              0                NaN     Curated
3                 116              0                NaN     Curated
4                  50              0                NaN     Curated
...               ...            ...                ...         ...
338               104             63           1.535617     Factory
339               559            197           1.367723  Playground
340               189             89           1.211730  Playground
342                 1              0                NaN     Factory
343               158            126           1.427594     Factory

[331 rows x 4 columns]


,averageTradeWithin2hr,minTradeWithin2hr,maxTradeWithin2hr,medianTradeWithin2hr
projectType,,,,
Curated,198.625000,2,427,202.0
Factory,63.740741,1,542,23.5
Playground,95.769231,1,349,47.0


In [309]:
view = mints[mints["projectid"] == 337]